<table width="100%">
    <tr style="border-bottom:solid 2pt #009EE3">
        <td style="text-align:left" width="10%">
            <a href="unit_conversion_ecg.dwipynb" download><img src="../../images/icons/download.png"></a>
        </td>
        <td style="text-align:left" width="10%">
            <a href="https://mybinder.org/v2/gh/biosignalsnotebooks/biosignalsnotebooks/biosignalsnotebooks_binder?filepath=biosignalsnotebooks_environment%2Fcategories%2FPre-Process%2Funit_conversion_ecg.dwipynb" target="_blank"><img src="../../images/icons/program.png" title="Be creative and test your solutions !"></a>
        </td>
        <td></td>
        <td style="text-align:left" width="5%">
            <a href="../MainFiles/biosignalsnotebooks.ipynb"><img src="../../images/icons/home.png"></a>
        </td>
        <td style="text-align:left" width="5%">
            <a href="../MainFiles/contacts.ipynb"><img src="../../images/icons/contacts.png"></a>
        </td>
        <td style="text-align:left" width="5%">
            <a href="https://github.com/biosignalsnotebooks/biosignalsnotebooks" target="_blank"><img src="../../images/icons/github.png"></a>
        </td>
        <td style="border-left:solid 2pt #009EE3" width="15%">
            <img src="../../images/ost_logo.png">
        </td>
    </tr>
</table>

<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_4"><div id="image_img" class="header_image_4"></div></td>
        <td class="header_text">ECG Sensor - Unit Conversion </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">pre-process&#9729;ecg&#9729;conversion</td> 
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

The <strong><span class="color2">OpenSignals</span></strong> outputted file formats contain raw data, so each sample has a digital unit.

In scientific terms it is recommended the use of specific units, like electric tension (V) or electric current (A).
Each sensor that *Plux* commercialise has a datasheet where a transfer function is mentioned for unit conversion be done.

The next lines are intended to explain how this conversion can be programmatically done.

In spite of the unit conversion procedure has some common steps applicable to all sensors, the current <strong><span class="color5">Jupyter Notebook</span></strong> is dedicated to the unit conversion procedure of signals acquired with ECG sensor.

<hr>

<p class="steps">1 - Importation of the needed packages</p>

In [1]:
# biosignalsnotebooks Python package with useful functions that support and complement the available Notebooks
import biosignalsnotebooks as bsnb

# Function used for creating a numpy array, where a mathematical operation can be applied to each entry in 
# an easy and automatic way. On the other side, linspace, here will be used for generation of a time-axis.
from numpy import array, linspace

<p class="steps">2 - Download of the sensor datasheet (from <a href="http://www.biosignalsplux.com/en/learn/documentation">http://www.biosignalsplux.com/en/learn/documentation</a>).</p>
In this case we are working with ECG, being our file located at <a href="http://www.biosignalsplux.com/datasheets/ECG_Sensor_Datasheet.pdf">http://www.biosignalsplux.com/datasheets/ECG_Sensor_Datasheet.pdf</a>

In [2]:
# Embedding of .pdf file
from IPython.display import IFrame
IFrame(src="https://www.biosignalsplux.com/datasheets/ECG_Sensor_Datasheet.pdf#page=2", width="100%", height="350")

<p class="steps">3 - Extraction of the transfer function from the beginning of the second page</p> 

\begin{equation}
ECG_{mV} = \frac{(\frac{ADC}{2^n} - \frac{1}{2}).VCC}{G_{ECG}}
\end{equation}

<table width="100%">
    <tr>
        <td width="33%" style="text-align:left;vertical-align:top">$ECG_{mV}$ - ECG value in mV</td>
        <td width="33%" style="text-align:left;vertical-align:top">$ADC$ - Digital value sampled from the channel (initialism of "Analog to Digital Converter")</td>
        <td width="33%" style="text-align:left;vertical-align:top">$n$ - Number of bits of the channel (dependent on the chosen resolution specified on <span class="color2">OpenSignals</span> previously to the acquisition stage [8, 12 or 16 bits])</td>
    </tr>
    <tr>
        <td style="text-align:left;vertical-align:top">$VCC$ - 3000 mV</td>
        <td style="text-align:left;vertical-align:top">$G_{ECG}$ - 1000</td>
        <td></td>
    </tr>
</table>

<p class="steps">4 - Loading of data stored in <strong><span class="color2">biosignalsnotebooks</span></strong> own signal library</p> 

In [3]:
# Data loading
data, header = bsnb.load_signal("ecg_sample", get_header=True)

In the following cell, some relevant information is stored inside variables. This relevant information includes the mac-address of the device, channel number and signal acquisition parameters such as resolution and sampling rate.

For a detailed explanation of how to access this info, the <a href="../Load/signal_loading_preparatory_steps.ipynb" target="_blank">"Signal Loading - Working with File Header"<img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> Notebook should be consulted.

In [4]:
ch = "CH1" # Channel
sr = 200 # Sampling rate
resolution = 16 # Resolution (number of available bits)

Access to acquired signal samples and storage inside a new variable.

In [5]:
signal = data[ch]

<p class="steps">5 - Final unit conversion (to <span class="color5">mV</span>) by applying the transfer function sample by sample</p>
Definition of $VCC$ and $G_{ECG}$ constants

In [6]:
vcc = 3000 # mV
gain = 1000

In [7]:
signal_mv = (((array(signal) / 2**resolution) - 0.5) * vcc) / gain

<p class="steps">6 - Time axis generation</p>

In [8]:
time = bsnb.generate_time(signal_mv, sr)

Comparison between RAW and mV signal.

In [9]:
bsnb.plot([time, time], [signal, signal_mv], y_axis_label=["Raw Data", "Electric Voltage (mV)"],
          grid_lines=1, grid_columns=2, grid_plot=True)

<span class="color2">Similar Notebooks</span>, dedicated to the unit conversion of other sensors, are available in the following "conversion" section of <a href="../MainFiles/by_tag.ipynb">Notebooks Grouped by Tag page <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>

A list of Plux&#39;s available sensors (and the respective datasheets) can be accessed at <a href="http://www.biosignalsplux.com/en/products/sensors">Sensors page <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>.

<strong><span class="color7">We hope that you have enjoyed this guide. </span><span class="color2">biosignalsnotebooks</span><span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining <a href="../MainFiles/biosignalsnotebooks.ipynb">Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></span></strong> ! 

<hr>
<table width="100%">
    <tr>
        <td style="border-right:solid 3px #009EE3" width="20%">
            <img src="../../images/ost_logo.png">
        </td>
        <td width="40%" style="text-align:left">
            <a href="../MainFiles/aux_files/biosignalsnotebooks_presentation.pdf" target="_blank">&#9740; Project Presentation</a>
            <br>
            <a href="https://github.com/biosignalsnotebooks/biosignalsnotebooks" target="_blank">&#9740; GitHub Repository</a>
            <br>
            <a href="https://pypi.org/project/biosignalsnotebooks/" target="_blank">&#9740; How to install biosignalsnotebooks Python package ?</a>
            <br>
            <a href="../MainFiles/signal_samples.ipynb">&#9740; Signal Library</a>
        </td>
        <td width="40%" style="text-align:left">
            <a href="../MainFiles/biosignalsnotebooks.ipynb">&#9740; Notebook Categories</a>
            <br>
            <a href="../MainFiles/by_diff.ipynb">&#9740; Notebooks by Difficulty</a>
            <br>
            <a href="../MainFiles/by_signal_type.ipynb">&#9740; Notebooks by Signal Type</a>
            <br>
            <a href="../MainFiles/by_tag.ipynb">&#9740; Notebooks by Tag</a>
        </td>
    </tr>
</table>

In [10]:
from biosignalsnotebooks.__notebook_support__ import css_style_apply
css_style_apply()

.................... CSS Style Applied to Jupyter Notebook .........................


In [11]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>